# Working with BFL FLUX in Pixeltable

This notebook demonstrates how to use Black Forest Labs (BFL) FLUX models for image generation and editing through Pixeltable.

[BFL FLUX](https://docs.bfl.ai/) offers state-of-the-art text-to-image generation with models like FLUX.2 and FLUX 1.1, featuring:
- High-fidelity image generation with accurate hands, faces, and textures
- Multi-reference image editing
- Precise hex color control
- Typography and text rendering

## Prerequisites

1. A BFL API key from [dashboard.bfl.ai](https://dashboard.bfl.ai)
2. Pixeltable installed


## Setup

First, install Pixeltable and set up your API key:


In [ ]:
# If running from the pixeltable repository:
# %pip install -e /path/to/pixeltable
# Otherwise:
# %pip install pixeltable


In [ ]:
import os

# Set your BFL API key (or set BFL_API_KEY environment variable)
# os.environ['BFL_API_KEY'] = 'your-api-key-here'


In [ ]:
import pixeltable as pxt
from pixeltable.functions import bfl

# Create a directory for our examples
pxt.drop_dir('bfl_demo', force=True)
pxt.create_dir('bfl_demo')


## Text-to-Image Generation

Generate images from text prompts using FLUX models. FLUX 1.1 [pro] offers fast, reliable results.


In [ ]:
# Create a table for image generation
prompts_t = pxt.create_table(
    'bfl_demo.prompts',
    {'prompt': pxt.String, 'style': pxt.String},
    if_exists='replace'
)

# Add a computed column that generates images from prompts
prompts_t.add_computed_column(
    image=bfl.generate(
        prompts_t.prompt,
        model='flux-pro-1.1',
        width=1024,
        height=1024
    )
)


In [ ]:
# Insert some prompts to generate images
prompts_t.insert([
    {'prompt': 'A majestic mountain landscape at golden hour, photorealistic', 'style': 'landscape'},
    {'prompt': 'A futuristic city with flying cars and neon lights, cyberpunk style', 'style': 'sci-fi'},
])


In [ ]:
# View the generated images
prompts_t.select(prompts_t.prompt, prompts_t.image).show()


## Available Models

BFL offers several FLUX models with different capabilities:

| Model | Best For | Speed |
|-------|----------|-------|
| `flux-2-pro` | Production workflows, best balance | < 10s |
| `flux-2-max` | Highest quality, grounding search | < 15s |
| `flux-2-flex` | Fine-grained control (steps/guidance) | Variable |
| `flux-pro-1.1` | Fast, reliable baseline | Fast |
| `flux-pro-1.1-ultra` | High resolution | Fast |
| `flux-dev` | Development (non-commercial) | Free |


## Image Editing

Edit existing images with text prompts using FLUX models. This is powerful for:
- Changing backgrounds
- Adding or removing objects
- Style transfer
- Multi-reference editing


In [ ]:
# Create a table for image editing
edit_t = pxt.create_table(
    'bfl_demo.edits',
    {
        'original': pxt.Image,
        'edit_prompt': pxt.String
    },
    if_exists='replace'
)

# Add computed column for edited images
edit_t.add_computed_column(
    edited=bfl.edit(
        edit_t.edit_prompt,
        edit_t.original,
        model='flux-2-pro'
    )
)


In [ ]:
# Insert an image and edit prompt
# Replace with your own image URL or path
edit_t.insert(
    original='https://images.unsplash.com/photo-1506744038136-46273834b3fb?w=1024',
    edit_prompt='Add a dramatic sunset with vibrant orange and purple colors'
)


In [ ]:
# View original and edited images
edit_t.select(edit_t.original, edit_t.edited, edit_t.edit_prompt).show()


## Using Seeds for Reproducibility

Use the `seed` parameter to get reproducible results:


In [ ]:
# Create a table with seed support
seed_t = pxt.create_table(
    'bfl_demo.seeded',
    {'prompt': pxt.String, 'seed': pxt.Int},
    if_exists='replace'
)

seed_t.add_computed_column(
    image=bfl.generate(
        seed_t.prompt,
        model='flux-pro-1.1',
        width=512,
        height=512,
        seed=seed_t.seed
    )
)

# Same seed = same image
seed_t.insert([
    {'prompt': 'A red rose in a crystal vase', 'seed': 42},
    {'prompt': 'A red rose in a crystal vase', 'seed': 42},  # Same result
    {'prompt': 'A red rose in a crystal vase', 'seed': 123},  # Different result
])


## Best Practices

1. **Use specific prompts**: Include details about style, lighting, composition, and subject
2. **Start with flux-pro-1.1**: Fast and reliable for most use cases
3. **Use seeds for reproducibility**: When you need consistent results
4. **Resolution**: Minimum 64x64, max 4MP (2048x2048), dimensions must be multiples of 16
5. **Safety tolerance**: Default is 2; lower = stricter moderation (0-5 scale)

## Learn More

- [BFL Documentation](https://docs.bfl.ai/)
- [FLUX.2 Overview](https://docs.bfl.ai/flux_2/flux2_overview)
- [Text-to-Image API](https://docs.bfl.ai/flux_2/flux2_text_to_image)
- [Image Editing API](https://docs.bfl.ai/flux_2/flux2_image_editing)
- [Pricing](https://docs.bfl.ai/quick_start/pricing)
